# Load Dataset

In [3]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [4]:
#!unzip -q "/content/MyDrive/MyDrive/UMHackathon/UMHackathon 2021 Finance Dataset.zip" -d "dataset" 
!unzip -q "/content/MyDrive/MyDrive/UMHackathon 2021 Finance Dataset.zip" -d "dataset" 

In [44]:
from google.colab import files
uploaded=files.upload()

Saving BPAMERS20201130-TEST.xlsx to BPAMERS20201130-TEST.xlsx


In [5]:
!pip install selenium -q
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 904 kB 9.1 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelea

In [6]:
import pandas as pd
import glob
import os
import dateutil.parser
import functools
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import datetime
import random

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from dateutil.relativedelta import relativedelta
import requests
from bs4 import BeautifulSoup

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data Cleaning

In [86]:
dataset_dir = "/content/dataset/BPAM Evaluated Prices Rated Range Prices (LT)"

# Merge both files (BPANERP & BPAMERS) together as a single pandas frame
def read_all_csvs(dataset_dir):
  sub_dirs = [f for f in os.listdir(dataset_dir) if not os.path.isfile(f)]
  df_BPAMERP = pd.DataFrame()
  df_BPAMERS = pd.DataFrame()
  df_joined = pd.DataFrame()
  for sub_dir in sub_dirs:
    full_path = os.path.join(dataset_dir, sub_dir)
    for file_name in os.listdir(full_path):
      full_file_name = os.path.join(full_path, file_name)
      if "BPAMERP" in file_name:
        temp_df_BPAMERP =  pd.read_csv(full_file_name, parse_dates=True)
        df_BPAMERP = pd.concat([df_BPAMERP, temp_df_BPAMERP])
      elif "BPAMERS" in file_name:
        temp_df_BPAMERS =  pd.read_csv(full_file_name, parse_dates=True)
        df_BPAMERS = pd.concat([df_BPAMERS, temp_df_BPAMERS])
    df_joined = pd.concat([df_joined, temp_df_BPAMERP.merge(temp_df_BPAMERS, on = ['STOCK CODE','ISIN CODE','STOCK NAME'], how = 'inner')])
  return df_BPAMERP.reset_index(drop=True), df_BPAMERS.reset_index(drop=True), df_joined.sort_values(by=['VALUE DATE']).reset_index(drop=True)

df_BPAMERP, df_BPAMERS, df_joined = read_all_csvs(dataset_dir)
df_joined.info(memory_usage='deep')

# parse datetime data type
date_columns = [column for column in df_joined.columns if "DATE" in column]
for date_column in date_columns:
  df_joined[date_column] = pd.to_datetime(df_joined[date_column], infer_datetime_format=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34206 entries, 0 to 34205
Data columns (total 49 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   STOCK CODE                                34206 non-null  object 
 1   ISIN CODE                                 34206 non-null  object 
 2   STOCK NAME                                34206 non-null  object 
 3   VALUE DATE                                34206 non-null  object 
 4   EVAL UPPER THRESHOLD YIELD                34206 non-null  float64
 5   EVAL MID YIELD                            34206 non-null  float64
 6   EVAL LOWER THRESHOLD YIELD                34206 non-null  float64
 7   EVAL LOWER THRESHOLD PRICE                34206 non-null  float64
 8   EVAL MID PRICE                            34206 non-null  float64
 9   EVAL UPPER THRESHOLD PRICE                34206 non-null  float64
 10  MODIFIED DURATION                 

In [88]:
#New test set

temp_df_BPAMERP =  pd.read_csv("/content/BPAMERP20201130-TEST.csv", parse_dates=True)
temp_df_BPAMERS =  pd.read_csv("/content/BPAMERS20201130-TEST.csv", parse_dates=True)
temp_df_joined = pd.DataFrame()
temp_df_joined = pd.concat([temp_df_joined, temp_df_BPAMERP.merge(temp_df_BPAMERS, on = ['STOCK CODE','ISIN CODE','STOCK NAME'], how = 'inner')])
date_columns = [column for column in temp_df_joined.columns if "DATE" in column]
for date_column in date_columns:
  temp_df_joined[date_column] = pd.to_datetime(temp_df_joined[date_column], infer_datetime_format=True)  
from dateutil.relativedelta import relativedelta
#Fixed test set date mistake
temp_df_joined.loc[temp_df_joined["STOCK CODE"]=="VZ170398","MATURITY DATE"]=temp_df_joined.loc[temp_df_joined["STOCK CODE"]=="VZ170398","MATURITY DATE"].apply(lambda x: x+relativedelta(years=100))
temp_df_joined
df_joined=pd.concat([df_joined,temp_df_joined],axis=0)
df_joined=df_joined.sort_values(by=['VALUE DATE']).reset_index(drop=True)
df_joined.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34216 entries, 0 to 34215
Data columns (total 49 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   STOCK CODE                                34216 non-null  object        
 1   ISIN CODE                                 34216 non-null  object        
 2   STOCK NAME                                34216 non-null  object        
 3   VALUE DATE                                34216 non-null  datetime64[ns]
 4   EVAL UPPER THRESHOLD YIELD                34216 non-null  float64       
 5   EVAL MID YIELD                            34216 non-null  float64       
 6   EVAL LOWER THRESHOLD YIELD                34216 non-null  float64       
 7   EVAL LOWER THRESHOLD PRICE                34216 non-null  float64       
 8   EVAL MID PRICE                            34216 non-null  float64       
 9   EVAL UPPER THRESHOLD PRICE  

In [89]:
# parse bool data type
d = {'Y': True, 'N': False}
df_joined["CALLABLE/PUTTABLE"] = df_joined["CALLABLE/PUTTABLE"].map(d)
df_joined["CONVERTIBLE/EXCHANGABLE"] = df_joined["CONVERTIBLE/EXCHANGABLE"].map(d)

# parse category data type
obj_columns = df_joined.select_dtypes(include='object').columns
for obj_column in obj_columns:
  df_joined[obj_column] = df_joined[obj_column].astype('category')

# show the memory usage
df_joined.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34216 entries, 0 to 34215
Data columns (total 49 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   STOCK CODE                                34216 non-null  category      
 1   ISIN CODE                                 34216 non-null  category      
 2   STOCK NAME                                34216 non-null  category      
 3   VALUE DATE                                34216 non-null  datetime64[ns]
 4   EVAL UPPER THRESHOLD YIELD                34216 non-null  float64       
 5   EVAL MID YIELD                            34216 non-null  float64       
 6   EVAL LOWER THRESHOLD YIELD                34216 non-null  float64       
 7   EVAL LOWER THRESHOLD PRICE                34216 non-null  float64       
 8   EVAL MID PRICE                            34216 non-null  float64       
 9   EVAL UPPER THRESHOLD PRICE  

In [91]:
# check null value
df_joined.isnull().sum().sort_values()

STOCK CODE                                      0
PRINCIPLE                                       0
BOND TYPE                                       0
BOND CLASS                                      0
ISSUE DATE                                      0
MATURITY DATE                                   0
DAY COUNT BASIS                                 0
FACILITY AMOUNT/FACILITY LIMIT(MYR MIL)         0
ISSUER NAME                                     0
FACILITY OUTSTANDING AMOUNT(MYR MIL)            0
BOND CURRENT OUTSTANDING AMOUNT(MYR MIL)        0
REMAINING TENURE                                0
ISSUER FACILITY LIMIT(MYR MIL)                  0
ISSUER OUTSTANDING AMOUNT(MYR MIL)              0
SECTOR                                          0
CALLABLE/PUTTABLE                               0
CONVERTIBLE/EXCHANGABLE                         0
BOND ISSUE AMOUNT(MYR MIL)                      0
FACILITY CODE                                   0
RATING                                          0


In [92]:
# impute null value
df_joined["COUPON FREQUENCY"].fillna(0, inplace = True)
df_joined["NEXT COUPON RATE"].fillna(0, inplace = True)
df_joined["PREVIOUS COUPON RATE"].fillna(0, inplace = True)
df_joined["ISLAMIC CONCEPT"] = df_joined["ISLAMIC CONCEPT"].cat.add_categories("CONVENTIONAL").fillna("CONVENTIONAL")
df_joined["RATING AGENCY"] = df_joined["RATING AGENCY"].cat.add_categories("NOT RATED").fillna("NOT RATED")
df_joined["RATING ACTION"] = df_joined["RATING ACTION"].cat.add_categories("NOT RATED").fillna("NOT RATED")

In [93]:
# check null value
df_joined.isnull().sum().sort_values()

STOCK CODE                                      0
RATING ACTION                                   0
RATING AGENCY                                   0
ISSUE DATE                                      0
MATURITY DATE                                   0
DAY COUNT BASIS                                 0
COUPON FREQUENCY                                0
PREVIOUS COUPON RATE                            0
NEXT COUPON RATE                                0
FACILITY AMOUNT/FACILITY LIMIT(MYR MIL)         0
FACILITY OUTSTANDING AMOUNT(MYR MIL)            0
BOND ISSUE AMOUNT(MYR MIL)                      0
BOND CURRENT OUTSTANDING AMOUNT(MYR MIL)        0
REMAINING TENURE                                0
ISSUER FACILITY LIMIT(MYR MIL)                  0
ISSUER OUTSTANDING AMOUNT(MYR MIL)              0
ISLAMIC CONCEPT                                 0
SECTOR                                          0
CALLABLE/PUTTABLE                               0
CONVERTIBLE/EXCHANGABLE                         0


# Features Engineering

In [94]:
# extract month data from [VALUE DATE] 
# eg: VALUE DATE 2020-04-01 refers to March, that is why we need to subtract by 1
df_joined["VALUE DATE MONTH"] = df_joined["VALUE DATE"].dt.to_period('M') - 1
df_joined["VALUE DATE MONTH"]

0        2019-07
1        2019-07
2        2019-07
3        2019-07
4        2019-07
          ...   
34211    2020-11
34212    2020-11
34213    2020-11
34214    2020-11
34215    2020-11
Name: VALUE DATE MONTH, Length: 34216, dtype: period[M]

In [ ]:
df_joined["REMAINING TENURE IN DAYS"]=(df_joined["MATURITY DATE"]-df_joined["VALUE DATE"]).dt.days
df_joined

In [95]:
# calculate maturity duration in days 
df_joined["MATURITY DURATION"] = (df_joined["MATURITY DATE"] - df_joined["ISSUE DATE"]).dt.days
df_joined["MATURITY DURATION"]

0        5114
1        6573
2        6209
3        5844
4        5479
         ... 
34211    6210
34212    3653
34213    5660
34214    5296
34215    1826
Name: MATURITY DURATION, Length: 34216, dtype: int64

In [96]:
# calculate accured interest
# assume ACTACT and ACTBOTH are the same
def cal_accured_interest(bond):
  prev_date = bond["PREVIOUS PAYMENT DATE"] if bond["PREVIOUS PAYMENT DATE"] is not pd.NaT else bond["ISSUE DATE"]
  num_of_days_since_last_coupon_payment = float((bond["VALUE DATE"] - prev_date).days)
  if bond["DAY COUNT BASIS"] in ["ACTACT","ACTBOTH"]:
    payment_period = float((bond["NEXT PAYMENT DATE"] - prev_date).days)
    if bond["COUPON FREQUENCY"] == 0:
      return 0
    accured_interest = 100 * (bond["NEXT COUPON RATE"] / 100) / bond["COUPON FREQUENCY"] * num_of_days_since_last_coupon_payment / payment_period
    return accured_interest
  elif bond["DAY COUNT BASIS"] in ["ACT365"]:
    accured_interest = 100 * (bond["NEXT COUPON RATE"] / 100) * num_of_days_since_last_coupon_payment / 365
    return accured_interest

df_joined["ACCRUED INTEREST"] = df_joined.apply(cal_accured_interest, axis=1)
df_joined["ACCRUED INTEREST"]

0        0.258630
1        1.699726
2        1.673973
3        1.648219
4        1.622466
           ...   
34211    1.825297
34212    2.012027
34213    0.511740
34214    2.951507
34215    0.845753
Name: ACCRUED INTEREST, Length: 34216, dtype: float64

In [97]:
def opr_extract():
  URL = "https://www.bnm.gov.my/opr-decision-and-statement"
  page = requests.get(URL)
  soup = BeautifulSoup(page.content,'lxml')
  content=soup.find_all('td')[4:]
  rates=[]
  dates=[]
  for i in range(len(content)):
    if i%4==0:
      dates.append(str(content[i]).replace("<td>","").replace("</td>",""))
    elif i%2==0:
      rates.append(float(str(content[i]).replace("<td>\n\t\t","").replace("\n\t</td>","")))
    else:
      continue
  i=0
  while i in range(len(rates)-1):
    if rates[i] == rates[i+1]:
        del rates[i]
        del dates[i]
    else:
        i += 1
  OPR_movement={}
  for i in range(len(rates)):
    OPR_movement[dates[i]]=rates[i]
  return OPR_movement

def assign_opr(value_date):
  for (date, opr) in OPR_movement: 
    date = dateutil.parser.parse(date) 
    if value_date >= date:
      return opr

# set opr movement
OPR_movement = opr_extract()
# sort the opr dict according to the date in descending order
OPR_movement = sorted(OPR_movement.items(), key=lambda item: dateutil.parser.parse(item[0]), reverse=True)
df_joined["OPR MOVEMENT"] = df_joined["VALUE DATE"].apply(assign_opr)
df_joined["OPR MOVEMENT"]

0        3.00
1        3.00
2        3.00
3        3.00
4        3.00
         ... 
34211    1.75
34212    1.75
34213    1.75
34214    1.75
34215    1.75
Name: OPR MOVEMENT, Length: 34216, dtype: float64

In [98]:
def inflation_rate_extract():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument("start-maximized")
  chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
  chrome_options.add_experimental_option('useAutomationExtension', False)

  wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  wd.get("https://www.fxempire.com/macro/malaysia/inflation-rate")

  #Most recent three years
  wd.find_element_by_xpath('//div[@class=" css-16ycfp3"]').click()
  wd.find_element_by_xpath("//div[@class=' css-14rzzno-menu']//*[name()='div']").click()

  wd.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(wd, 20).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='recharts-wrapper']"))))
  elements = WebDriverWait(wd, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@class='recharts-wrapper']//*[name()='svg']//*[name()='g' and @class='recharts-layer recharts-bar']//*[name()='g']//*[name()='g']//*[name()='g']")))
  ir=[]
  for element in elements:
    ActionChains(wd).move_to_element(element).perform()
    mouseover = WebDriverWait(wd, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='recharts-wrapper']")))
    ir.append(float(mouseover.text.strip().replace("%","")[-6:].replace("\n","")))
  current_date=datetime.date.today()-relativedelta(days=datetime.date.today().day)
  inflation_dict={}
  for i in range(len(ir)-1,-1,-1):
    inflation_dict[str(current_date)]=ir[i]
    current_date-=relativedelta(days=current_date.day)
  inflation_dict['2021-06-30']=3.4
  inflation_dict['2021-05-31']=4.4
  return inflation_dict

def assign_ir(value_date):
  for (date, ir) in inflation_dict: 
    date = dateutil.parser.parse(date) 
    if value_date >= date:
      return ir

#set inflation rate
inflation_dict=inflation_rate_extract()
# sort the ir dict according to the date in descending order
inflation_dict = sorted(inflation_dict.items(), key=lambda item: dateutil.parser.parse(item[0]), reverse=True)
df_joined["INFLATION RATE"]=df_joined["VALUE DATE"].apply(assign_ir)
df_joined

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,REMAINING TENURE IN DAYS,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR MOVEMENT,INFLATION RATE
0,VR120013,MYBVR1200136,PLUS BERHAD IMTN 4.720% 12.01.2026 - Series 1 ...,2019-08-01,3.787,3.753,3.719,105.289,105.488,105.688,5.540,36.073,0.000,0.000,0.000,-0.003,-0.003,-0.002,2.3,201200003,Projek Lebuhraya Usahasama Berhad,ISLAMIC,Fixed Rate Bond,Corporate,AAA IS,MARC,2012-01-12,2026-01-12,NaT,ACT365,2.0,2012-07-12,2019-07-12,4.720,2020-01-13,4.720,23350,18900,1000,1000,7Y,34350,29900,MUSYARAKAH,INFRASTRUCTURES AND UTILITIES,2019-03-25,False,False,Affirm,2356,2019-07,5114,0.258630,3.00,1.5
1,VV180161,MYBVV1801612,UITM SOLAR IMTN 6.600% 25.04.2036,2019-08-01,5.669,5.613,5.557,109.971,110.614,111.262,10.263,144.188,-0.010,-0.010,-0.010,0.114,0.115,0.115,0.0,201800020,UiTM Solar Power Sdn Berhad,ISLAMIC,Fixed Rate Bond,Corporate,AA- IS,MARC,2018-04-27,2036-04-25,NaT,ACT365,2.0,2018-10-29,2019-04-29,6.600,2019-10-29,6.600,240,222,17,17,20Y,240,222,MURABAHAH,INFRASTRUCTURES AND UTILITIES,2019-01-09,False,False,Affirm,6112,2019-07,6573,1.699726,3.00,1.5
2,VU180162,MYBVU1801622,UITM SOLAR IMTN 6.500% 27.04.2035,2019-08-01,5.567,5.511,5.455,109.689,110.311,110.937,9.951,133.599,-0.010,-0.010,-0.010,0.109,0.110,0.111,0.0,201800020,UiTM Solar Power Sdn Berhad,ISLAMIC,Fixed Rate Bond,Corporate,AA- IS,MARC,2018-04-27,2035-04-27,NaT,ACT365,2.0,2018-10-29,2019-04-29,6.500,2019-10-29,6.500,240,222,15,15,20Y,240,222,MURABAHAH,INFRASTRUCTURES AND UTILITIES,2019-01-09,False,False,Affirm,5748,2019-07,6209,1.673973,3.00,1.5
3,VT180163,MYBVT1801632,UITM SOLAR IMTN 6.400% 27.04.2034,2019-08-01,5.472,5.416,5.360,109.299,109.896,110.498,9.605,122.675,-0.005,-0.005,-0.005,0.053,0.052,0.052,0.0,201800020,UiTM Solar Power Sdn Berhad,ISLAMIC,Fixed Rate Bond,Corporate,AA- IS,MARC,2018-04-27,2034-04-27,NaT,ACT365,2.0,2018-10-29,2019-04-29,6.400,2019-10-29,6.400,240,222,10,10,15Y,240,222,MURABAHAH,INFRASTRUCTURES AND UTILITIES,2019-01-09,False,False,Affirm,5383,2019-07,5844,1.648219,3.00,1.5
4,VS180164,MYBVS1801642,UITM SOLAR IMTN 6.300% 27.04.2033,2019-08-01,5.392,5.346,5.300,108.726,109.194,109.666,9.219,111.461,0.000,0.000,0.000,-0.001,-0.002,-0.001,2.3,201800020,UiTM Solar Power Sdn Berhad,ISLAMIC,Fixed Rate Bond,Corporate,AA- IS,MARC,2018-04-27,2033-04-27,NaT,ACT365,2.0,2018-10-29,2019-04-29,6.300,2019-10-29,6.300,240,222,15,15,15Y,240,222,MURABAHAH,INFRASTRUCTURES AND UTILITIES,2019-01-09,False,False,Affirm,5018,2019-07,5479,1.622466,3.00,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34211,PU110040,MYBPU1100407,BESRAYA 5.360% 28.07.2028,2020-12-01,3.542,3.473,3.404,112.095,112.588,113.083,6.257,47.198,0.009,0.009,0.009,-0.068,-0.068,-0.069,1.7,201100048,Besraya (M) Sdn Berhad,ISLAMIC,Fixed Rate Bond,Corporate,AA3,RAM,2011-07-28,2028-07-28,NaT,ACTACT,2.0,2012-01-30,2020-07-28,5.3

In [99]:
def mgs_extract():
  value_dates=df_joined["VALUE DATE"].unique()
  conv_MGS_movement={}
  islam_MGS_movement={}
  for date in value_dates:
    date=str(date)[:10]
    prev_date=datetime.date(int(date[0:4]),int(date[5:7]),int(date[8:10]))-relativedelta(days=1)
    URL = "https://www.bnm.gov.my/government-securities-yield?p_p_id=my_gov_bnm_yield_display_portlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_my_gov_bnm_yield_display_portlet_tradingDateTxt="+str(prev_date)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content,'html.parser')
    dates=[]
    mgs=[]
    content=soup.find_all('td')[11:43]
    #Get from earlier dates in the case of weekends/public holidays
    while(str(content[5]).strip().replace('<td align="center">\n','').replace('</td>','').replace('\n','').replace(' ','')==''):
      prev_date-=relativedelta(days=1)
      URL = "https://www.bnm.gov.my/government-securities-yield?p_p_id=my_gov_bnm_yield_display_portlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_my_gov_bnm_yield_display_portlet_tradingDateTxt="+str(prev_date)
      page = requests.get(URL)
      soup = BeautifulSoup(page.content,'html.parser')
      content=soup.find_all('td')[11:43]
    for i in range(len(content)):
      if i%8==5:
        mgs.append(float(str(content[i]).strip().replace('<td align="center">\n','').replace('</td>','').replace('\n','').replace(' ','').replace('*','')))
    conv_MGS_movement[date]={}
    conv_MGS_movement[date]["3-year"]=mgs[0]
    conv_MGS_movement[date]["5-year"]=mgs[1]
    conv_MGS_movement[date]["7-year"]=mgs[2]
    conv_MGS_movement[date]["10-year"]=mgs[3]
    content=soup.find_all('td')[53:80]
    mgs=[]
    for i in range(len(content)):
      if i%7==4:
        mgs.append(float(str(content[i]).strip().replace('<td align="center">\n','').replace('</td>','').replace('\n','').replace(' ','').replace('*','')))
    islam_MGS_movement[date]={}
    islam_MGS_movement[date]["3-year"]=mgs[0]
    islam_MGS_movement[date]["5-year"]=mgs[1]
    islam_MGS_movement[date]["7-year"]=mgs[2]
    islam_MGS_movement[date]["10-year"]=mgs[3]
  return conv_MGS_movement,islam_MGS_movement

def assign_mgs(islamic,value_date,remaining_tenure):
  date=str(value_date)[:10]
  mgs_dict=conv_MGS_movement
  if islamic:
    mgs_dict=islam_MGS_movement
  if remaining_tenure in ["3M","6M","1Y","2Y","3Y"]:
    return mgs_dict[date]["3-year"]
  elif remaining_tenure=="5Y":
    return mgs_dict[date]["5-year"]
  elif remaining_tenure=="&Y":
    return mgs_dict[date]["7-year"]
  return mgs_dict[date]["10-year"]

#set mgs movement
conv_MGS_movement, islam_MGS_movement=mgs_extract()
df_joined["PRINCIPLE"]=df_joined["PRINCIPLE"].replace("CONVENTIONAL",0).replace("ISLAMIC",1)
df_joined["MGS"]=df_joined[["PRINCIPLE","VALUE DATE","REMAINING TENURE"]].apply(lambda x: assign_mgs(x["PRINCIPLE"],x["VALUE DATE"],x["REMAINING TENURE"]),axis=1)
df_joined["CREDIT SPREAD"]=df_joined["EVAL MID YIELD"]-df_joined["MGS"]
df_joined

,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,REMAINING TENURE IN DAYS,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR MOVEMENT,INFLATION RATE,MGS,CREDIT SPREAD
0,VR120013,MYBVR1200136,PLUS BERHAD IMTN 4.720% 12.01.2026 - Series 1 ...,2019-08-01,3.787,3.753,3.719,105.289,105.488,105.688,5.540,36.073,0.000,0.000,0.000,-0.003,-0.003,-0.002,2.3,201200003,Projek Lebuhraya Usahasama Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2012-01-12,2026-01-12,NaT,ACT365,2.0,2012-07-12,2019-07-12,4.720,2020-01-13,4.720,23350,18900,1000,1000,7Y,34350,29900,MUSYARAKAH,INFRASTRUCTURES AND UTILITIES,2019-03-25,False,False,Affirm,2356,2019-07,5114,0.258630,3.00,1.5,3.61,0.143
1,VV180161,MYBVV1801612,UITM SOLAR IMTN 6.600% 25.04.2036,2019-08-01,5.669,5.613,5.557,109.971,110.614,111.262,10.263,144.188,-0.010,-0.010,-0.010,0.114,0.115,0.115,0.0,201800020,UiTM Solar Power Sdn Berhad,1,Fixed Rate Bond,Corporate,AA- IS,MARC,2018-04-27,2036-04-25,NaT,ACT365,2.0,2018-10-29,2019-04-29,6.600,2019-10-29,6.600,240,222,17,17,20Y,240,222,MURABAHAH,INFRASTRUCTURES AND UTILITIES,2019-01-09,False,False,Affirm,6112,2019-07,6573,1.699726,3.00,1.5,3.61,2.003
2,VU180162,MYBVU1801622,UITM SOLAR IMTN 6.500% 27.04.2035,2019-08-01,5.567,5.511,5.455,109.689,110.311,110.937,9.951,133.599,-0.010,-0.010,-0.010,0.109,0.110,0.111,0.0,201800020,UiTM Solar Power Sdn Berhad,1,Fixed Rate Bond,Corporate,AA- IS,MARC,2018-04-27,2035-04-27,NaT,ACT365,2.0,2018-10-29,2019-04-29,6.500,2019-10-29,6.500,240,222,15,15,20Y,240,222,MURABAHAH,INFRASTRUCTURES AND UTILITIES,2019-01-09,False,False,Affirm,5748,2019-07,6209,1.673973,3.00,1.5,3.61,1.901
3,VT180163,MYBVT1801632,UITM SOLAR IMTN 6.400% 27.04.2034,2019-08-01,5.472,5.416,5.360,109.299,109.896,110.498,9.605,122.675,-0.005,-0.005,-0.005,0.053,0.052,0.052,0.0,201800020,UiTM Solar Power Sdn Berhad,1,Fixed Rate Bond,Corporate,AA- IS,MARC,2018-04-27,2034-04-27,NaT,ACT365,2.0,2018-10-29,2019-04-29,6.400,2019-10-29,6.400,240,222,10,10,15Y,240,222,MURABAHAH,INFRASTRUCTURES AND UTILITIES,2019-01-09,False,False,Affirm,5383,2019-07,5844,1.648219,3.00,1.5,3.61,1.806
4,VS180164,MYBVS1801642,UITM SOLAR IMTN 6.300% 27.04.2033,2019-08-01,5.392,5.346,5.300,108.726,109.194,109.666,9.219,111.461,0.000,0.000,0.000,-0.001,-0.002,-0.001,2.3,201800020,UiTM Solar Power Sdn Berhad,1,Fixed Rate Bond,Corporate,AA- IS,MARC,2018-04-27,2033-04-27,NaT,ACT365,2.0,2018-10-29,2019-04-29,6.300,2019-10-29,6.300,240,222,15,15,15Y,240,222,MURABAHAH,INFRASTRUCTURES AND UTILITIES,2019-01-09,False,False,Affirm,5018,2019-07,5479,1.622466,3.00,1.5,3.61,1.736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34211,PU110040,MYBPU1100407,BESRAYA 5.360% 28.07.2028,2020-12-01,3.542,3.473,3.404,112.095,112.588,113.083,6.257,47.198,0.009,0.009,0.009,-0.068,-0.068,-0.069,1.7,201100048,Besraya (M) Sdn Berhad,1,Fixed Rate Bond,Corporate,AA3,RAM,2011-07-28,2028-0

In [100]:
# calculate: 
# 1. changes In EVAL MID Price/EVAL LOWER THRESHOLD PRICE/EVAL UPPER THRESHOLD PRICE	
# 2. changes in EVAL MID YIELD/EVAL LOWER THRESHOLD YIELD/EVAL UPPER THRESHOLD YIELD
# 3. NEXT MONTH CHANGES IN EVAL MID PRICE
def transform_eval_mid_price(df_joined):
  temp_df_joined = pd.DataFrame()
  for stock_code in df_joined["STOCK CODE"].unique():
    temp_df = df_joined[df_joined["STOCK CODE"] == stock_code].sort_values("VALUE DATE MONTH")
    # calculate changes In EVAL MID Price
    temp_df["CHANGES IN EVAL MID PRICE"] = temp_df["EVAL MID PRICE"].diff()
    temp_df["CHANGES IN EVAL LOWER THRESHOLD PRICE"] = temp_df["EVAL LOWER THRESHOLD PRICE"].diff()
    temp_df["CHANGES IN EVAL UPPER THRESHOLD PRICE"] = temp_df["EVAL UPPER THRESHOLD PRICE"].diff()
    # calculate changes In EVAL YIELD Price
    temp_df["CHANGES IN EVAL MID YIELD"] = temp_df["EVAL MID YIELD"].diff()
    temp_df["CHANGES IN EVAL LOWER THRESHOLD YIELD"] = temp_df["EVAL LOWER THRESHOLD YIELD"].diff()
    temp_df["CHANGES IN EVAL UPPER THRESHOLD YIELD"] = temp_df["EVAL UPPER THRESHOLD YIELD"].diff()
    # calculate moving average changes In EVAL MID Price for the past 5 months
    # temp_df["MA-5 CHANGES IN EVAL MID PRICE"] = temp_df["CHANGES IN EVAL MID PRICE"].rolling(5,1).mean()
    # calculate moving average changes In EVAL MID Yield for the past 5 months
    # temp_df["MA-5 CHANGES IN EVAL MID YIELD"] = temp_df["CHANGES IN EVAL MID YIELD"].rolling(5,1).mean()
    # shift the NEXT MONTH CHANGES IN EVAL MID PRICE to current month 
    temp_df["NEXT MONTH CHANGES IN EVAL MID PRICE"] = temp_df["CHANGES IN EVAL MID PRICE"].shift(-1)
    temp_df_joined = pd.concat([temp_df_joined, temp_df])
    
  return temp_df_joined

df_joined = transform_eval_mid_price(df_joined)
df_joined

,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,REMAINING TENURE IN DAYS,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR MOVEMENT,INFLATION RATE,MGS,CREDIT SPREAD,CHANGES IN EVAL MID PRICE,CHANGES IN EVAL LOWER THRESHOLD PRICE,CHANGES IN EVAL UPPER THRESHOLD PRICE,CHANGES IN EVAL MID YIELD,CHANGES IN EVAL LOWER THRESHOLD YIELD,CHANGES IN EVAL UPPER THRESHOLD YIELD,NEXT MONTH CHANGES IN EVAL MID PRICE
0,VR120013,MYBVR1200136,PLUS BERHAD IMTN 4.720% 12.01.2026 - Series 1 ...,2019-08-01,3.787,3.753,3.719,105.289,105.488,105.688,5.540,36.073,0.000,0.000,0.000,-0.003,-0.003,-0.002,2.3,201200003,Projek Lebuhraya Usahasama Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2012-01-12,2026-01-12,NaT,ACT365,2.0,2012-07-12,2019-07-12,4.72,2020-01-13,4.720,23350,18900,1000,1000,7Y,34350,29900,MUSYARAKAH,INFRASTRUCTURES AND UTILITIES,2019-03-25,False,False,Affirm,2356,2019-07,5114,0.258630,3.00,1.5,3.61,0.143,NaN,NaN,NaN,NaN,NaN,NaN,1.694
2696,VR120013,MYBVR1200136,PLUS BERHAD IMTN 4.720% 12.01.2026 - Series 1 ...,2019-09-03,3.484,3.451,3.418,106.987,107.182,107.377,5.468,35.224,-0.010,-0.010,-0.010,0.048,0.048,0.048,2.7,201200003,Projek Lebuhraya Usahasama Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2012-01-12,2026-01-12,NaT,ACT365,2.0,2012-07-12,2019-07-12,4.72,2020-01-13,4.720,23350,18900,1000,1000,7Y,34350,29900,MUSYARAKAH,INFRASTRUCTURES AND UTILITIES,2019-03-25,False,False,Affirm,2323,2019-08,5114,0.685370,3.00,1.4,3.36,0.091,1.694,1.698,1.689,-0.302,-0.301,-0.303,-1.215
4769,VR120013,MYBVR1200136,PLUS BERHAD IMTN 4.720% 12.01.2026 - Series 1 ...,2019-10-01,3.673,3.647,3.621,105.818,105.967,106.117,5.382,34.260,-0.011,-0.010,-0.009,0.061,0.055,0.049,3.0,201200003,Projek Lebuhraya Usahasama Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2012-01-12,2026-01-12,NaT,ACT365,2.0,2012-07-12,2019-07-12,4.72,2020-01-13,4.720,23350,18900,1000,1000,7Y,34350,29900,MUSYARAKAH,INFRASTRUCTURES AND UTILITIES,2019-03-25,False,False,Affirm,2295,2019-09,5114,1.047452,3.00,1.5,3.40,0.247,-1.215,-1.169,-1.260,0.196,0.203,0.189,-0.407
7790,VR120013,MYBVR1200136,PLUS BERHAD IMTN 4.720% 12.01.2026 - Series 1 ...,2019-11-01,3.737,3.706,3.675,105.385,105.560,105.736,5.295,33.296,-0.009,-0.010,-0.011,0.049,0.054,0.060,2.3,201200003,Projek Lebuhraya Usahasama Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2012-01-12,2026-01-12,NaT,ACT365,2.0,2012-07-12,2019-07-12,4.72,2020-01-13,4.720,23350,18900,1000,1000,7Y,34350,29900,MUSYARAKAH,INFRASTRUCTURES AND UTILITIES,2019-03-25,False,False,Affirm,2264,2019-10,5114,1.448329,3.00,1.1,3.47,0.236,-0.407,-0.433,-0.381,0.059,0.054,0.064,0.051
9762,VR120013,MYBVR1200136,PLUS BERHAD IMTN 4.720% 12.01.2026 - Series 1 ...,2019-12-02,3.724,3.685,3.646,105.393,105.611,105.830,5.213,32.390,-0.002,0.000,0.002,0.005,-0.007,-0.018,2.0,201200003,Projek Lebuhraya Usahasama Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2012-01-12,2026-01-12,NaT,ACT365,2.0,2012-07-12,2019-07-12,4.72,2020-01-13,4.720,23350,18900,1000,1000,7Y,34350,29900,MUSYARAKAH,INFRASTRUCTURES AND UTILITIES,2019-03-25,F

In [106]:
import math
# calculate remaining payments
def cal_remaining_payments(bond):
  if bond["DAY COUNT BASIS"] in ["ACTACT","ACTBOTH"]:
    if bond["COUPON FREQUENCY"] == 0:
      return 0
    remaining=bond["REMAINING TENURE IN DAYS"]*bond["COUPON FREQUENCY"]/365.25
    return math.ceil(remaining)
  elif bond["DAY COUNT BASIS"] in ["ACT365"]:
    remaining=bond["REMAINING TENURE IN DAYS"]*bond["COUPON FREQUENCY"]/365.25
    return math.ceil(remaining)

df_joined["REMAINING PAYMENTS"] = df_joined.apply(cal_remaining_payments, axis=1)
df_joined["REMAINING PAYMENTS"]
df_joined

,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,REMAINING TENURE IN DAYS,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR MOVEMENT,INFLATION RATE,MGS,CREDIT SPREAD,CHANGES IN EVAL MID PRICE,CHANGES IN EVAL LOWER THRESHOLD PRICE,CHANGES IN EVAL UPPER THRESHOLD PRICE,CHANGES IN EVAL MID YIELD,CHANGES IN EVAL LOWER THRESHOLD YIELD,CHANGES IN EVAL UPPER THRESHOLD YIELD,NEXT MONTH CHANGES IN EVAL MID PRICE,REMAINING PAYMENTS
0,VR120013,MYBVR1200136,PLUS BERHAD IMTN 4.720% 12.01.2026 - Series 1 ...,2019-08-01,3.787,3.753,3.719,105.289,105.488,105.688,5.540,36.073,0.000,0.000,0.000,-0.003,-0.003,-0.002,2.3,201200003,Projek Lebuhraya Usahasama Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2012-01-12,2026-01-12,NaT,ACT365,2.0,2012-07-12,2019-07-12,4.72,2020-01-13,4.72,23350,18900,1000,1000,7Y,34350,29900,MUSYARAKAH,INFRASTRUCTURES AND UTILITIES,2019-03-25,False,False,Affirm,2356,2019-07,5114,0.258630,3.00,1.5,3.61,0.143,NaN,NaN,NaN,NaN,NaN,NaN,1.694,13
1232,VI150054,MYBVI1500542,UEMS IMTN 4.58% 10.04.2020 - Issue No. 5,2019-08-01,3.661,3.615,3.569,100.616,100.647,100.679,0.667,0.778,0.000,0.000,0.000,-0.002,-0.003,-0.002,2.7,201200093,UEM Sunrise Berhad,1,Fixed Rate Bond,Corporate,AA- IS,MARC,2015-04-10,2020-04-10,NaT,ACT365,2.0,2015-10-12,2019-04-10,4.58,2019-10-10,4.58,2000,800,150,150,1Y,4000,2600,MURABAHAH,PROPERTY AND REAL ESTATE,2019-02-21,False,False,Affirm,253,2019-07,1827,1.417918,3.00,1.5,3.31,0.305,NaN,NaN,NaN,NaN,NaN,NaN,-0.007,2
192,VL170301,MYBVL1703017,QSPS Green SRI Sukuk 5.400% 06.10.2025 - T14,2019-08-01,5.213,5.108,5.003,100.969,101.523,102.080,5.125,31.703,0.000,0.000,0.000,0.000,0.000,-0.001,3.0,201700065,Quantum Solar Park (Semenanjung) Sdn Berhad,1,Fixed Rate Bond,Corporate(G),A+ IS,MARC,2017-10-06,2025-10-06,NaT,ACT365,2.0,2018-04-06,2019-04-08,5.40,2019-10-07,5.40,1000,980,30,30,7Y,1000,980,MURABAHAH + TAWARRUQ,INFRASTRUCTURES AND UTILITIES,2019-07-23,False,False,Affirm,2258,2019-07,2922,1.701370,3.00,1.5,3.61,1.498,NaN,NaN,NaN,NaN,NaN,NaN,1.162,13
1231,PN120073,MYBPN1200731,PRASARANA SUKUK MURABAHAH 3.77% 06.09.2022 - T1,2019-08-01,3.447,3.419,3.391,100.938,101.020,101.102,2.859,9.938,0.000,0.000,0.000,-0.001,-0.001,-0.001,2.0,201200067,Prasarana Malaysia Berhad,1,Fixed Rate Bond,Quasi-Govt,NR(LT),NOT RATED,2012-09-06,2022-09-06,NaT,ACT365,2.0,2013-03-06,2019-03-06,3.77,2019-09-06,3.77,2000,2000,1000,1000,5Y,46936,29500,MURABAHAH,TRANSPORTATION,NaT,False,False,NOT RATED,1132,2019-07,3652,1.528658,3.00,1.5,3.43,-0.011,NaN,NaN,NaN,NaN,NaN,NaN,0.422,7
1230,VN120337,MYBVN1203378,KLK IMTN 4.00% 02.09.2022 - Issue No. 1,2019-08-01,3.701,3.658,3.615,100.861,100.986,101.111,2.834,9.801,-0.002,0.000,0.002,0.005,-0.001,-0.007,2.3,201200064,Kuala Lumpur Kepong Berhad,1,Fixed Rate Bond,Corporate,AA1,RAM,2012-09-03,2022-09-02,NaT,ACT365,2.0,2013-03-04,2019-03-04,4.00,2019-09-03,4.00,1000,1000,1000,1000,5Y,3400,2600,IJARAH,PLANTATION AND AGRICULTURE,2018-09-21,False,False,Reaffirm,1128,2019-07,3651,1.643836,3.00,1.5,3.43,0.228,NaN,NaN,NaN,NaN,NaN,NaN,0.536,7
...,...,...,...,.

In [108]:
# remove the first row and the last row of each stocks
# df_joined.dropna(subset=["CHANGES IN EVAL MID PRICE","NEXT MONTH CHANGES IN EVAL MID PRICE"],inplace=True)
df_joined.dropna(subset=["CHANGES IN EVAL MID PRICE"],inplace=True)
df_joined

,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,REMAINING TENURE IN DAYS,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR MOVEMENT,INFLATION RATE,MGS,CREDIT SPREAD,CHANGES IN EVAL MID PRICE,CHANGES IN EVAL LOWER THRESHOLD PRICE,CHANGES IN EVAL UPPER THRESHOLD PRICE,CHANGES IN EVAL MID YIELD,CHANGES IN EVAL LOWER THRESHOLD YIELD,CHANGES IN EVAL UPPER THRESHOLD YIELD,NEXT MONTH CHANGES IN EVAL MID PRICE,REMAINING PAYMENTS
2952,VO130172,MYBVO1301725,KIMANIS IMTN 4.850% 14.12.2023 - Tranche No 30,2019-09-03,3.900,3.840,3.780,103.704,103.944,104.185,3.811,17.334,0.005,0.000,-0.005,-0.029,-0.010,0.011,2.7,201200054,Kimanis Power Sdn Berhad,1,Fixed Rate Bond,Corporate,AA- IS,MARC,2013-06-14,2023-12-14,NaT,ACT365,2.0,2013-12-16,2019-06-14,4.85,2019-12-16,4.85,1160,750,10,10,5Y,1160,750,IJARAH + ISTISNA,INFRASTRUCTURES AND UTILITIES,2018-09-14,False,False,Affirm,1563,2019-08,3835,1.076301,3.00,1.4,3.25,0.590,0.670,0.616,0.724,-0.184,-0.198,-0.170,-0.242,9
2159,VN120176,MYBVN1201760,AMAN IMTN 4.130% 12.04.2022 - Tranche No. 16,2019-09-03,3.355,3.317,3.279,101.915,102.011,102.106,2.420,7.284,-0.010,-0.010,-0.010,0.017,0.018,0.017,1.7,201100006,Aman Sukuk Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2012-04-12,2022-04-12,NaT,ACT365,2.0,2012-10-12,2019-04-12,4.13,2019-10-14,4.13,10000,4250,355,355,3Y,10000,4250,MUSYARAKAH,PROPERTY AND REAL ESTATE,2019-08-20,False,False,Affirm,952,2019-08,3652,1.629370,3.00,1.4,3.13,0.187,0.423,0.428,0.418,-0.189,-0.188,-0.190,-0.269,6
2937,VZ180333,MYBVZ1803338,DANAINFRA IMTN 5.170% 26.11.2048 - Tranche No 84,2019-09-03,3.910,3.881,3.852,121.827,122.407,122.991,16.207,374.233,0.000,0.000,0.000,-0.004,-0.004,-0.005,2.3,201200042,DanaInfra Nasional Berhad,1,Fixed Rate Bond,Quasi-Govt,NR(LT),NOT RATED,2018-11-26,2048-11-26,NaT,ACT365,2.0,2019-05-27,2019-05-27,5.17,2019-11-26,5.17,61000,52020,755,755,25Y+,74000,57520,MURABAHAH,FINANCIAL SERVICES,NaT,False,False,NOT RATED,10677,2019-08,10958,1.402274,3.00,1.4,3.36,0.521,7.756,7.687,7.825,-0.406,-0.407,-0.405,-3.333,59
2163,VN130094,MYBVN1300943,AMAN IMTN 4.100% 08.03.2023 - Tranche No 28,2019-09-03,3.381,3.353,3.325,102.360,102.454,102.547,3.190,12.328,-0.010,-0.010,-0.010,0.026,0.027,0.026,3.3,201100006,Aman Sukuk Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2013-03-08,2023-03-08,NaT,ACT365,2.0,2013-09-09,2019-03-08,4.10,2019-09-10,4.10,10000,4250,160,160,5Y,10000,4250,MUSYARAKAH,PROPERTY AND REAL ESTATE,2019-08-20,False,False,Affirm,1282,2019-08,3652,2.010685,3.00,1.4,3.25,0.103,0.732,0.736,0.727,-0.232,-0.231,-0.233,-0.441,8
3783,VS180153,MYBVS1801535,DANAINFRA IMTN 4.900% 29.04.2033 - Tranche 3,2019-09-03,3.626,3.602,3.578,113.626,113.905,114.184,10.046,126.054,0.002,0.002,0.002,-0.032,-0.031,-0.032,3.7,201700072,DanaInfra Nasional Berhad,1,Fixed Rate Bond,Quasi-Govt,NR(LT),NOT RATED,2018-05-02,2033-04-29,NaT,ACT365,2.0,2018-11-02,2019-05-02,4.90,2019-11-04,4.90,13000,5500,700,700,15Y,74000,57520,MURABAHAH,FINANCIAL SERVICES,NaT,False,False,NOT RATED,4987,2019-08,5476,1.664658,3.00,1.4,3.36,0.242,3.175,3.1

In [109]:
df_joined.isnull().sum().sort_values().tail(10)

STOCK NAME                                  0
ISIN CODE                                   0
COMPOSITE LIQUIDITY SCORE (T-1)             0
EVAL UPPER THRESHOLD YIELD                  0
NEXT PAYMENT DATE                         291
FIRST PAYMENT DATE                       1785
PREVIOUS PAYMENT DATE                    2019
NEXT MONTH CHANGES IN EVAL MID PRICE     2427
RATING EFFECTIVE DATE                    6508
EXPECTED MATURITY DATE                  29340
dtype: int64

# Train Test Split

In [110]:
# filter pandas dataframe to AA & AAA bonds only
df_joined = df_joined[df_joined["RATING"].str.startswith("AA")]
df_joined

,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,REMAINING TENURE IN DAYS,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR MOVEMENT,INFLATION RATE,MGS,CREDIT SPREAD,CHANGES IN EVAL MID PRICE,CHANGES IN EVAL LOWER THRESHOLD PRICE,CHANGES IN EVAL UPPER THRESHOLD PRICE,CHANGES IN EVAL MID YIELD,CHANGES IN EVAL LOWER THRESHOLD YIELD,CHANGES IN EVAL UPPER THRESHOLD YIELD,NEXT MONTH CHANGES IN EVAL MID PRICE,REMAINING PAYMENTS
2952,VO130172,MYBVO1301725,KIMANIS IMTN 4.850% 14.12.2023 - Tranche No 30,2019-09-03,3.900,3.840,3.780,103.704,103.944,104.185,3.811,17.334,0.005,0.000,-0.005,-0.029,-0.010,0.011,2.7,201200054,Kimanis Power Sdn Berhad,1,Fixed Rate Bond,Corporate,AA- IS,MARC,2013-06-14,2023-12-14,NaT,ACT365,2.0,2013-12-16,2019-06-14,4.85,2019-12-16,4.85,1160,750,10,10,5Y,1160,750,IJARAH + ISTISNA,INFRASTRUCTURES AND UTILITIES,2018-09-14,False,False,Affirm,1563,2019-08,3835,1.076301,3.00,1.4,3.25,0.590,0.670,0.616,0.724,-0.184,-0.198,-0.170,-0.242,9
2159,VN120176,MYBVN1201760,AMAN IMTN 4.130% 12.04.2022 - Tranche No. 16,2019-09-03,3.355,3.317,3.279,101.915,102.011,102.106,2.420,7.284,-0.010,-0.010,-0.010,0.017,0.018,0.017,1.7,201100006,Aman Sukuk Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2012-04-12,2022-04-12,NaT,ACT365,2.0,2012-10-12,2019-04-12,4.13,2019-10-14,4.13,10000,4250,355,355,3Y,10000,4250,MUSYARAKAH,PROPERTY AND REAL ESTATE,2019-08-20,False,False,Affirm,952,2019-08,3652,1.629370,3.00,1.4,3.13,0.187,0.423,0.428,0.418,-0.189,-0.188,-0.190,-0.269,6
2163,VN130094,MYBVN1300943,AMAN IMTN 4.100% 08.03.2023 - Tranche No 28,2019-09-03,3.381,3.353,3.325,102.360,102.454,102.547,3.190,12.328,-0.010,-0.010,-0.010,0.026,0.027,0.026,3.3,201100006,Aman Sukuk Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2013-03-08,2023-03-08,NaT,ACT365,2.0,2013-09-09,2019-03-08,4.10,2019-09-10,4.10,10000,4250,160,160,5Y,10000,4250,MUSYARAKAH,PROPERTY AND REAL ESTATE,2019-08-20,False,False,Affirm,1282,2019-08,3652,2.010685,3.00,1.4,3.25,0.103,0.732,0.736,0.727,-0.232,-0.231,-0.233,-0.441,8
2953,VP120300,MYBVP1203001,KIMANIS IMTN 5.100% 08.08.2024 - Tranche No. 9,2019-09-03,3.924,3.854,3.784,105.221,105.542,105.864,4.338,22.268,0.005,0.000,-0.005,-0.034,-0.011,0.011,0.0,201200054,Kimanis Power Sdn Berhad,1,Fixed Rate Bond,Corporate,AA- IS,MARC,2012-08-08,2024-08-08,NaT,ACT365,2.0,2013-02-08,2019-08-08,5.10,2020-02-11,5.10,1160,750,70,70,5Y,1160,750,IJARAH + ISTISNA,INFRASTRUCTURES AND UTILITIES,2018-09-14,False,False,Affirm,1801,2019-08,4383,0.363288,3.00,1.4,3.25,0.604,0.786,0.724,0.848,-0.190,-0.204,-0.176,-0.325,10
2164,VP110402,MYBVP1104027,AMAN IMTN 4.250% 20.10.2023 - Tranche No. 11,2019-09-03,3.415,3.382,3.349,103.186,103.314,103.443,3.715,16.480,-0.010,-0.010,-0.010,0.031,0.031,0.031,2.7,201100006,Aman Sukuk Berhad,1,Fixed Rate Bond,Corporate,AAA IS,MARC,2011-10-21,2023-10-20,NaT,ACT365,2.0,2012-04-23,2019-04-22,4.25,2019-10-21,4.25,10000,4250,205,205,5Y,10000,4250,MUSYARAKAH,PROPERTY AND REAL ESTATE,2019-08-20,False,False,Affirm,1508,2019-08,4382,1.560274,3.00,1.4,3.25,0.132,0.961,0.987,0.937

In [111]:
# test size 
train_months = df_joined["VALUE DATE MONTH"].unique()[:-1]
test_months_no_grd_truth = df_joined["VALUE DATE MONTH"].unique()[-1:]

<PeriodArray>
['2020-11']
Length: 1, dtype: period[M]

In [112]:
train_dataset = df_joined[df_joined["VALUE DATE MONTH"].isin(train_months)]
train_dataset.dropna(subset=["NEXT MONTH CHANGES IN EVAL MID PRICE"],inplace=True)
test_dataset_no_grd_truth = df_joined[df_joined["VALUE DATE MONTH"].isin(test_months_no_grd_truth)]

print("Number of train set: "+ str(len(train_dataset)))
print("Number of test_dataset_no_grd_truth: "+ str(len(test_dataset_no_grd_truth)))

Number of train set: 20791
Number of test_dataset_no_grd_truth: 10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [113]:
train_dataset["VALUE DATE"].value_counts()

2020-02-03    1521
2020-03-02    1512
2020-04-01    1511
2020-05-04    1500
2020-10-01    1497
2020-09-01    1494
2020-06-01    1488
2020-07-01    1485
2020-08-03    1479
2020-01-02    1477
2019-10-01    1468
2019-09-03    1461
2019-12-02    1455
2019-11-01    1433
2020-11-02      10
Name: VALUE DATE, dtype: int64

In [114]:
train_dataset.to_csv("engineered_train_dataset_AA_AAA_bonds_09092021.csv", index=False)
test_dataset_no_grd_truth.to_csv("engineered_test_dataset_AA_AAA_bonds_no_grd_truth_09092021.csv", index=False)